In [73]:
# import cv2
# import numpy as np
# from shapely.geometry import Polygon, MultiPolygon, LineString
# from shapely.ops import cascaded_union
# import random
# import math
# import networkx as nx

# no_of_houses = 20
# no_of_valleys = 2
# no_of_hills = 3
# house_width, house_height = 20, 20
# building_width, building_height = random.randint(30, 60), random.randint(30, 60)
# hill_radius = 30
# valley_radius = 20
# building_labels = ["Hospital", "Gas Station", "Public Park", "School"]

# vertex_weights = {"House": 1, "Hospital": 2, "Gas Station": 3, "Public Park": 4, "School": 5}

# def is_complete(graph: nx.Graph) -> bool:
#     n = len(graph.nodes)
#     max_edges = (n * (n - 1)) // 2
#     return len(graph.edges) == max_edges

# def random_non_intersecting_polygon(existing_polygons, width, height):
#     while True:
#         x, y = random.randint(0, 512 - width), random.randint(0, 512 - height)
#         new_polygon = Polygon([(x, y), (x + width, y), (x + width, y + height), (x, y + height)])
        
#         intersects = False
#         for existing_polygon in existing_polygons:
#             if new_polygon.intersects(existing_polygon):
#                 intersects = True
#                 break
        
#         if not intersects:
#             return new_polygon

# # Initialize the image
# image = np.zeros((512, 512), dtype=np.uint8)

# # Draw a hill
# for _ in range(no_of_hills):
#     hill_center = (random.randint(hill_radius, 512 - hill_radius), random.randint(hill_radius, 512 - hill_radius))
#     cv2.circle(image, hill_center, hill_radius, 255, -1)
#     hill_polygon = Polygon([(hill_center[0] + hill_radius * np.cos(angle), hill_center[1] + hill_radius * np.sin(angle)) for angle in np.linspace(0, 2 * np.pi, 100)]).buffer(0)

# # Draw valleys
# valley_polygons = []
# for _ in range(no_of_valleys):
#     x, y = random.randint(0, 512 - 1), random.randint(0, 512 - 1)
#     valley_polygon = Polygon([(x, y), (x + 1, y), (x + 1, y + 1), (x, y + 1)]).buffer(valley_radius)
#     valley_polygons.append(valley_polygon)

#     for x, y in valley_polygon.exterior.coords:
#         cv2.circle(image, (int(x), int(y)), valley_radius, 255, -1)

# # Draw houses
# house_polygons = []
# for _ in range(no_of_houses):
#     house_polygon = random_non_intersecting_polygon(house_polygons + valley_polygons + [hill_polygon], house_width, house_height)
#     house_polygons.append(house_polygon)

#     x, y = int(house_polygon.bounds[0]), int(house_polygon.bounds[1])
#     cv2.rectangle(image, (x, y), (x + house_width, y + house_height), 255, -1)

# # Draw buildings with labels
# label_polygons = []
# for label in building_labels:
#     building_polygon = random_non_intersecting_polygon(house_polygons + valley_polygons + [hill_polygon], building_width, building_height)
#     label_polygons.append((building_polygon, label))

#     x, y = int(building_polygon.bounds[0]), int(building_polygon.bounds[1])
#     cv2.rectangle(image, (x, y), (x + building_width, y + building_height), 255, -1)
#     cv2.putText(image, label, (x, y - 5), cv2.FONT_HERSHEY_SIMPLEX, 0.5, 255, 1)

# def get_polygon_center(polygon):
#     x, y = np.array(polygon.centroid.coords).squeeze()
#     return int(x), int(y)

# # Extract center points of shapes
# center_points = []
# for polygon in house_polygons:
#     center_points.append(get_polygon_center(polygon))

# # Add center points for labeled buildings
# for polygon, label in label_polygons:
#     center_points.append(get_polygon_center(polygon))

# # Create a complete graph using the NetworkX library
# G = nx.complete_graph(len(center_points))

# # Set the node positions to the center points of the shapes
# pos = {i: center_points[i] for i in range(len(center_points))}
# nx.set_node_attributes(G, pos, "pos")

# # Draw rectilinear lines
# def draw_rectilinear_line(image, p1, p2, color, thickness):
#     x1, y1 = p1
#     x2, y2 = p2
#     cv2.line(image, (x1, y1), (x1, y2), color, thickness)
#     cv2.line(image, (x1, y2), (x2, y2), color, thickness)

# # Visualize the graph with rectilinear lines
# image_with_graph = image.copy()
# for i, j in G.edges():
#     draw_rectilinear_line(image_with_graph, center_points[i], center_points[j], 128, 1)

# # Calculate rectilinear distance between two points
# def rectilinear_distance(p1, p2):
#     return abs(p1[0] - p2[0]) + abs(p1[1] - p2[1])

# def edge_weight(p1, p2, hill_polygon, valley_polygons):
#     weight = rectilinear_distance(p1, p2)
#     line1 = LineString([p1, (p1[0], p2[1])])
#     line2 = LineString([(p1[0], p2[1]), p2])
    
#     if line1.intersects(hill_polygon) or line2.intersects(hill_polygon):
#         weight *= 2
    
#     if any(line1.intersects(valley) or line2.intersects(valley) for valley in valley_polygons):
#         weight *= 3

#     return weight

# # Add edge weights based on the distance and the presence of hills or valleys
# for i, j in G.edges():
#     G[i][j]['weight'] = edge_weight(center_points[i], center_points[j], hill_polygon, valley_polygons)

# # Visualize the graph with edge weights
# image_with_graph_weights = image.copy()
# for i, j in G.edges():
#     draw_rectilinear_line(image_with_graph_weights, center_points[i], center_points[j], 128, 1)
    
#     # Calculate the midpoint of the edge
#     midpoint = ((center_points[i][0] + center_points[j][0]) // 2, (center_points[i][1] + center_points[j][1]) // 2)
    
#     # Draw the edge weight as a text label
#     weight = int(G[i][j]['weight'])
#     cv2.putText(image_with_graph_weights, str(weight), midpoint, cv2.FONT_HERSHEY_SIMPLEX, 0.5, 255, 1)
    
# # Display the images
# cv2.imshow("Original Image", image)
# cv2.imshow("Image with Graph", image_with_graph)
# cv2.imshow("Image with Graph and Weights", image_with_graph_weights)
# cv2.waitKey(0)
# cv2.destroyAllWindows()

# # Save the images
# cv2.imwrite("original_image.png", image)
# cv2.imwrite("image_with_graph.png", image_with_graph)
# cv2.imwrite("image_with_graph_weights.png", image_with_graph_weights)


In [74]:
# import cv2
# import numpy as np
# from shapely.geometry import Polygon, MultiPolygon, LineString
# from shapely.ops import cascaded_union
# import random
# import math
# import networkx as nx

# no_of_houses = 20
# no_of_valleys = 2
# no_of_hills = 3
# house_width, house_height = 20, 20
# building_width, building_height = random.randint(30, 60), random.randint(30, 60)
# hill_radius = 30
# valley_radius = 20
# building_labels = ["Hospital", "Gas Station", "Public Park", "School"]

# vertex_weights = {"House": 1, "Hospital": 2, "Gas Station": 3, "Public Park": 4, "School": 5}

# def is_complete(graph: nx.Graph) -> bool:
#     n = len(graph.nodes)
#     max_edges = (n * (n - 1)) // 2
#     return len(graph.edges) == max_edges

# def random_non_intersecting_polygon(existing_polygons, width, height):
#     while True:
#         x, y = random.randint(0, 512 - width), random.randint(0, 512 - height)
#         new_polygon = Polygon([(x, y), (x + width, y), (x + width, y + height), (x, y + height)])
        
#         intersects = False
#         for existing_polygon in existing_polygons:
#             if new_polygon.intersects(existing_polygon):
#                 intersects = True
#                 break
        
#         if not intersects:
#             return new_polygon

# # Initialize the image
# image = np.zeros((512, 512), dtype=np.uint8)

# # Draw a hill
# for _ in range(no_of_hills):
#     hill_center = (random.randint(hill_radius, 512 - hill_radius), random.randint(hill_radius, 512 - hill_radius))
#     cv2.circle(image, hill_center, hill_radius, 255, -1)
#     hill_polygon = Polygon([(hill_center[0] + hill_radius * np.cos(angle), hill_center[1] + hill_radius * np.sin(angle)) for angle in np.linspace(0, 2 * np.pi, 100)]).buffer(0)

# # Draw valleys
# valley_polygons = []
# for _ in range(no_of_valleys):
#     x, y = random.randint(0, 512 - 1), random.randint(0, 512 - 1)
#     valley_polygon = Polygon([(x, y), (x + 1, y), (x + 1, y + 1), (x, y + 1)]).buffer(valley_radius)
#     valley_polygons.append(valley_polygon)

#     for x, y in valley_polygon.exterior.coords:
#         cv2.circle(image, (int(x), int(y)), valley_radius, 255, -1)

# # Draw houses
# house_polygons = []
# for _ in range(no_of_houses):
#     house_polygon = random_non_intersecting_polygon(house_polygons + valley_polygons + [hill_polygon], house_width, house_height)
#     house_polygons.append(house_polygon)

#     x, y = int(house_polygon.bounds[0]), int(house_polygon.bounds[1])
#     cv2.rectangle(image, (x, y), (x + house_width, y + house_height), 255, -1)

# # Draw buildings with labels
# label_polygons = []
# for label in building_labels:
#     building_polygon = random_non_intersecting_polygon(house_polygons + valley_polygons + [hill_polygon], building_width, building_height)
#     label_polygons.append((building_polygon, label))

#     x, y = int(building_polygon.bounds[0]), int(building_polygon.bounds[1])
#     cv2.rectangle(image, (x, y), (x + building_width, y + building_height), 255, -1)
#     cv2.putText(image, label, (x, y - 5), cv2.FONT_HERSHEY_SIMPLEX, 0.5, 255, 1)

# def get_polygon_center(polygon):
#     x, y = np.array(polygon.centroid.coords).squeeze()
#     return int(x), int(y)

# # Extract center points of shapes
# center_points = []
# for polygon in house_polygons:
#     center_points.append(get_polygon_center(polygon))

# # Add center points for labeled buildings
# for polygon, label in label_polygons:
#     center_points.append(get_polygon_center(polygon))

# # Create a complete graph using the NetworkX library
# G = nx.complete_graph(len(center_points))

# # Set the node positions to the center points of the shapes
# pos = {i: center_points[i] for i in range(len(center_points))}
# nx.set_node_attributes(G, pos, "pos")

# def line_intersects_polygon(line_start, line_end, polygon):
#     line = LineString([line_start, line_end])
#     return line.intersects(polygon)

# # Draw the edges of the graph using rectilinear distance
# for i, j in G.edges:
#     x1, y1 = pos[i]
#     x2, y2 = pos[j]
#     mid_x, mid_y = (x1 + x2) // 2, (y1 + y2) // 2

#     # Check if the line intersects any of the polygons
#     intersects = False
#     for polygon in house_polygons + valley_polygons + [hill_polygon]:
#         if line_intersects_polygon((x1, y1), (mid_x, mid_y), polygon) or line_intersects_polygon((mid_x, mid_y), (x2, y2), polygon):
#             intersects = True
#             break

#     if not intersects:
#         cv2.line(image, (x1, y1), (mid_x, mid_y), 255, 2)
#         cv2.line(image, (mid_x, mid_y), (x2, y2), 255, 2)

# # Draw the equidistant grid with thin lines
# grid_spacing = 25
# for x in range(0, 512, grid_spacing):
#     cv2.line(image, (x, 0), (x, 512), 128, 1)
# for y in range(0, 512, grid_spacing):
#     cv2.line(image, (0, y), (512, y), 128, 1)

# # Show the final image
# cv2.imshow("Map", image)
# cv2.waitKey(0)
# cv2.destroyAllWindows()



In [75]:
# import cv2
# import numpy as np
# from shapely.geometry import Polygon, MultiPolygon, LineString
# from shapely.ops import cascaded_union
# import random
# import math
# import networkx as nx

# no_of_houses = 20
# no_of_valleys = 2
# no_of_hills = 3
# house_width, house_height = 20, 20
# building_width, building_height = random.randint(30, 60), random.randint(30, 60)
# hill_radius = 30
# valley_radius = 20
# building_labels = ["Hospital", "Gas Station", "Public Park", "School"]

# vertex_weights = {"House": 1, "Hospital": 2, "Gas Station": 3, "Public Park": 4, "School": 5}

# def is_complete(graph: nx.Graph) -> bool:
#     n = len(graph.nodes)
#     max_edges = (n * (n - 1)) // 2
#     return len(graph.edges) == max_edges

# def random_non_intersecting_polygon(existing_polygons, width, height):
#     while True:
#         x, y = random.randint(0, 512 - width), random.randint(0, 512 - height)
#         new_polygon = Polygon([(x, y), (x + width, y), (x + width, y + height), (x, y + height)])
        
#         intersects = False
#         for existing_polygon in existing_polygons:
#             if new_polygon.intersects(existing_polygon):
#                 intersects = True
#                 break
        
#         if not intersects:
#             return new_polygon

# # Initialize the image
# image = np.zeros((512, 512), dtype=np.uint8)

# # Draw a hill
# for _ in range(no_of_hills):
#     hill_center = (random.randint(hill_radius, 512 - hill_radius), random.randint(hill_radius, 512 - hill_radius))
#     cv2.circle(image, hill_center, hill_radius, 255, -1)
#     hill_polygon = Polygon([(hill_center[0] + hill_radius * np.cos(angle), hill_center[1] + hill_radius * np.sin(angle)) for angle in np.linspace(0, 2 * np.pi, 100)]).buffer(0)

# # Draw valleys
# valley_polygons = []
# for _ in range(no_of_valleys):
#     x, y = random.randint(0, 512 - 1), random.randint(0, 512 - 1)
#     valley_polygon = Polygon([(x, y), (x + 1, y), (x + 1, y + 1), (x, y + 1)]).buffer(valley_radius)
#     valley_polygons.append(valley_polygon)

#     for x, y in valley_polygon.exterior.coords:
#         cv2.circle(image, (int(x), int(y)), valley_radius, 255, -1)

# # Draw houses
# house_polygons = []
# for _ in range(no_of_houses):
#     house_polygon = random_non_intersecting_polygon(house_polygons + valley_polygons + [hill_polygon], house_width, house_height)
#     house_polygons.append(house_polygon)

#     x, y = int(house_polygon.bounds[0]), int(house_polygon.bounds[1])
#     cv2.rectangle(image, (x, y), (x + house_width, y + house_height), 255, -1)

# # Draw buildings with labels
# label_polygons = []
# for label in building_labels:
#     building_polygon = random_non_intersecting_polygon(house_polygons + valley_polygons + [hill_polygon], building_width, building_height)
#     label_polygons.append((building_polygon, label))

#     x, y = int(building_polygon.bounds[0]), int(building_polygon.bounds[1])
#     cv2.rectangle(image, (x, y), (x + building_width, y + building_height), 255, -1)
#     cv2.putText(image, label, (x, y - 5), cv2.FONT_HERSHEY_SIMPLEX, 0.5, 255, 1)

# def get_polygon_center(polygon):
#     x, y = np.array(polygon.centroid.coords).squeeze()
#     return int(x), int(y)

# # Extract center points of shapes
# center_points = []
# for polygon in house_polygons:
#     center_points.append(get_polygon_center(polygon))

# # Add center points for labeled buildings
# for polygon, label in label_polygons:
#     center_points.append(get_polygon_center(polygon))

# # Create a complete graph using the NetworkX library
# G = nx.complete_graph(len(center_points))

# # Set the node positions to the center points of the shapes
# pos = {i: center_points[i] for i in range(len(center_points))}
# nx.set_node_attributes(G, pos, "pos")

# def line_intersects_polygon(line_start, line_end, polygon):
#     line = LineString([line_start, line_end])
#     return line.intersects(polygon)

# # Draw the edges of the graph using rectilinear distance
# for i, j in G.edges:
#     x1, y1 = pos[i]
#     x2, y2 = pos[j]
#     mid_x, mid_y = (x1 + x2) // 2, (y1 + y2) // 2

#     # Check if the line intersects any of the polygons
#     intersects = False
#     for polygon in house_polygons + valley_polygons + [hill_polygon]:
#         if line_intersects_polygon((x1, y1), (mid_x, mid_y), polygon) or line_intersects_polygon((mid_x, mid_y), (x2, y2), polygon):
#             intersects = True
#             break

#     if not intersects:
#         cv2.line(image, (x1, y1), (mid_x, mid_y), 255, 2)
#         cv2.line(image, (mid_x, mid_y), (x2, y2), 255, 2)

# # Draw the equidistant grid with thin lines
# grid_spacing = 25
# for x in range(0, 512, grid_spacing):
#     cv2.line(image, (x, 0), (x, 512), 128, 1)
# for y in range(0, 512, grid_spacing):
#     cv2.line(image, (0, y), (512, y), 128, 1)

# # Show the final image
# cv2.imshow("Map", image)
# cv2.waitKey(0)
# cv2.destroyAllWindows()


In [76]:
import cv2
import numpy as np
from shapely.geometry import Polygon, MultiPolygon, LineString
from shapely.ops import cascaded_union
import random
import math
import networkx as nx

no_of_houses = 20
no_of_valleys = 2
no_of_hills = 3
house_width, house_height = 20, 20
building_width, building_height = random.randint(30, 60), random.randint(30, 60)
hill_radius = 30
valley_radius = 20
building_labels = ["Hospital", "Gas Station", "Public Park", "School"]
vertex_weights = {"House": 1, "Hospital": 2, "Gas Station": 3, "Public Park": 4, "School": 5}
grid_cell_size = 30

def assign_vertex_weights(graph, house_polygons, label_polygons):
    for i, _ in enumerate(house_polygons):
        graph.nodes[i]['weight'] = vertex_weights['House']
    
    for j, (_, label) in enumerate(label_polygons, start=len(house_polygons)):
        graph.nodes[j]['weight'] = vertex_weights[label]

def is_complete(graph: nx.Graph) -> bool:
    n = len(graph.nodes)
    max_edges = (n * (n - 1)) // 2
    return len(graph.edges) == max_edges

def random_non_intersecting_polygon(existing_polygons, width, height):
    while True:
        x, y = random.randint(0, 512 - width), random.randint(0, 512 - height)
        new_polygon = Polygon([(x, y), (x + width, y), (x + width, y + height), (x, y + height)])
        
        intersects = False
        for existing_polygon in existing_polygons:
            if new_polygon.intersects(existing_polygon):
                intersects = True
                break
        
        if not intersects:
            return new_polygon

# Initialize the image
image = np.zeros((512, 512), dtype=np.uint8)

# Draw a hill
for _ in range(no_of_hills):
    hill_center = (random.randint(hill_radius, 512 - hill_radius), random.randint(hill_radius, 512 - hill_radius))
    cv2.circle(image, hill_center, hill_radius, 255, -1)
    hill_polygon = Polygon([(hill_center[0] + hill_radius * np.cos(angle), hill_center[1] + hill_radius * np.sin(angle)) for angle in np.linspace(0, 2 * np.pi, 100)]).buffer(0)

# Draw valleys
valley_polygons = []
for _ in range(no_of_valleys):
    x, y = random.randint(0, 512 - 1), random.randint(0, 512 - 1)
    valley_polygon = Polygon([(x, y), (x + 1, y), (x + 1, y + 1), (x, y + 1)]).buffer(valley_radius)
    valley_polygons.append(valley_polygon)

    for x, y in valley_polygon.exterior.coords:
        cv2.circle(image, (int(x), int(y)), valley_radius, 255, -1)

# Draw houses
house_polygons = []
for _ in range(no_of_houses):
    house_polygon = random_non_intersecting_polygon(house_polygons + valley_polygons + [hill_polygon], house_width, house_height)
    house_polygons.append(house_polygon)

    x, y = int(house_polygon.bounds[0]), int(house_polygon.bounds[1])
    cv2.rectangle(image, (x, y), (x + house_width, y + house_height), 255, -1)

# Draw buildings with labels
label_polygons = []
for label in building_labels:
    building_polygon = random_non_intersecting_polygon(house_polygons + valley_polygons + [hill_polygon], building_width, building_height)
    label_polygons.append((building_polygon, label))

    x, y = int(building_polygon.bounds[0]), int(building_polygon.bounds[1])
    cv2.rectangle(image, (x, y), (x + building_width, y + building_height), 255, -1)
    cv2.putText(image, label, (x, y - 5), cv2.FONT_HERSHEY_SIMPLEX, 0.5, 255, 1)

# def get_polygon_center(polygon):
#     x, y = np.array(polygon.centroid.coords).squeeze()
#     return int(x), int(y)


def get_polygon_center(polygon,grid_cell_size):
    x, y = np.array(polygon.centroid.coords).squeeze()
    x = round(x / grid_cell_size) * grid_cell_size
    y = round(y / grid_cell_size) * grid_cell_size
    return int(x), int(y)

# Extract center points of shapes
center_points = []
for polygon in house_polygons:
    center_points.append(get_polygon_center(polygon,grid_cell_size))

# Add center points for labeled buildings
for polygon, label in label_polygons:
    center_points.append(get_polygon_center(polygon,grid_cell_size))

# Create a complete graph using the NetworkX library
G = nx.complete_graph(len(center_points))

# Set the node positions to the center points of the shapes
pos = {i: center_points[i] for i in range(len(center_points))}
nx.set_node_attributes(G, pos, "pos")

assign_vertex_weights(G, house_polygons, label_polygons)

# Draw rectilinear lines
def draw_rectilinear_line(image, p1, p2, color, thickness):
    x1, y1 = p1
    x2, y2 = p2
    cv2.line(image, (x1, y1), (x1, y2), color, thickness)
    cv2.line(image, (x1, y2), (x2, y2), color, thickness)

# Visualize the graph with rectilinear lines
image_with_graph = image.copy()
for i, j in G.edges():
    draw_rectilinear_line(image_with_graph, center_points[i], center_points[j], 128, 1)

# Calculate rectilinear distance between two points
def rectilinear_distance(p1, p2):
    return abs(p1[0] - p2[0]) + abs(p1[1] - p2[1])

def edge_weight(p1, p2, hill_polygon, valley_polygons):
    weight = rectilinear_distance(p1, p2)
    line1 = LineString([p1, (p1[0], p2[1])])
    line2 = LineString([(p1[0], p2[1]), p2])
    
    if line1.intersects(hill_polygon) or line2.intersects(hill_polygon):
        weight *= 2
    
    if any(line1.intersects(valley) or line2.intersects(valley) for valley in valley_polygons):
        weight *= 3

    return weight

# Add edge weights based on the distance and the presence of hills or valleys
for i, j in G.edges():
    G[i][j]['weight'] = edge_weight(center_points[i], center_points[j], hill_polygon, valley_polygons)

# Visualize the graph with edge weights
image_with_graph_weights = image.copy()
for i, j in G.edges():
    draw_rectilinear_line(image_with_graph_weights, center_points[i], center_points[j], 255, 2)
    
    # Calculate the midpoint of the edge
    midpoint = ((center_points[i][0] + center_points[j][0]) // 2, (center_points[i][1] + center_points[j][1]) // 2)
    
    # Draw the edge weight as a text label
    weight = int(G[i][j]['weight'])
    cv2.putText(image_with_graph_weights, str(weight), midpoint, cv2.FONT_HERSHEY_SIMPLEX, 0.5, 255, 1)

# Draw the grid lines in a thin line
for x in range(0, 512, grid_cell_size):
    cv2.line(image_with_graph_weights, (x, 0), (x, 512), 128, 1)
    cv2.line(image_with_graph_weights, (0, x), (512, x), 128, 1)


# Draw the vertex weights as black text labels
for i in range(len(center_points)):
    x, y = center_points[i]
    # Draw the vertex weight as a black text label
    weight = int(G.nodes[i]['weight'])
    cv2.putText(image_with_graph_weights, str(weight), (x, y), cv2.FONT_HERSHEY_SIMPLEX, 0.5, 0, 1)


In [77]:
def kruskal_with_vertex_weights(graph):
    sorted_edges = sorted(graph.edges(data=True), key=lambda x: x[2]['weight'] + graph.nodes[x[0]]['weight'] + graph.nodes[x[1]]['weight'])
    mst = nx.Graph()

    for u, v, data in sorted_edges:
        if u not in mst or v not in mst or not nx.has_path(mst, u, v):
            mst.add_edge(u, v, weight=data['weight'])

    return mst

# Find the Minimum Spanning Tree
mst = kruskal_with_vertex_weights(G)

# Visualize the MST
image_with_mst = image.copy()

# Draw the grid lines in a thin line
for x in range(0, 512, grid_cell_size):
    cv2.line(image_with_mst, (x, 0), (x, 512), 128, 1)
    cv2.line(image_with_mst, (0, x), (512, x), 128, 1)


# Draw the rectilinear edges in a thick white line
for i, j in mst.edges():
    draw_rectilinear_line(image_with_mst, center_points[i], center_points[j], 255, 2)

    # Calculate the midpoint of the edge
    midpoint = ((center_points[i][0] + center_points[j][0]) // 2, (center_points[i][1] + center_points[j][1]) // 2)
    
    # Draw the edge weight as a white text label
    weight = int(mst[i][j]['weight'])
    cv2.putText(image_with_mst, str(weight), midpoint, cv2.FONT_HERSHEY_SIMPLEX, 0.5, 255, 1)


# Draw the vertex weights as black text labels
for i in range(len(center_points)):
    x, y = center_points[i]
    
    weight = int(G.nodes[i]['weight'])
    cv2.putText(image_with_mst, str(weight), (x, y), cv2.FONT_HERSHEY_SIMPLEX, 0.5, 0, 1)


In [78]:
def nearest_neighbor_algorithm(graph, start_node=None):
    if start_node is None:
        start_node = random.choice(list(graph.nodes()))

    unvisited = list(graph.nodes())
    unvisited.remove(start_node)
    tour = [start_node]

    current_node = start_node
    while unvisited:
        min_distance = float('inf')
        closest_node = None

        for node in unvisited:
            distance = graph[current_node][node]['weight']
            if distance < min_distance:
                min_distance = distance
                closest_node = node

        tour.append(closest_node)
        unvisited.remove(closest_node)
        current_node = closest_node

    # Connect the last node back to the start_node
    tour.append(start_node)

    return tour

# Find the TSP tour
tsp_tour = nearest_neighbor_algorithm(G)


# Visualize the TSP tour
image_with_tsp = image.copy()

# Draw the grid lines in a thin line
for x in range(0, 512, grid_cell_size):
    cv2.line(image_with_tsp, (x, 0), (x, 512), 128, 1)
    cv2.line(image_with_tsp, (0, x), (512, x), 128, 1)

# Draw the rectilinear edges in a thick white line
for i in range(len(tsp_tour) - 1):
    node1 = tsp_tour[i]
    node2 = tsp_tour[i + 1]
    draw_rectilinear_line(image_with_tsp, center_points[node1], center_points[node2], 255, 2)

# Draw the vertex weights as black text labels
for i in range(len(center_points)):
    x, y = center_points[i]
    
    weight = int(G.nodes[i]['weight'])
    cv2.putText(image_with_tsp, str(weight), (x, y), cv2.FONT_HERSHEY_SIMPLEX, 0.5, 0, 1)


print(is_complete(G))

True


In [79]:
def two_opt_algorithm(graph, initial_tour=None):
    nodes = list(graph.nodes)
    
    def tour_length(tour):
        return sum(graph[tour[i - 1]][tour[i]]['weight'] for i in range(len(tour)))

    def two_opt_swap(tour, i, j):
        return tour[:i] + tour[i:j + 1][::-1] + tour[j + 1:]

    if initial_tour is None:
        initial_tour = nodes

    current_tour = initial_tour
    current_length = tour_length(current_tour)

    improved = True
    while improved:
        improved = False

        for i in range(1, len(current_tour) - 2):
            for j in range(i + 1, len(current_tour) - 1):
                new_tour = two_opt_swap(current_tour, i, j)
                new_length = tour_length(new_tour)

                if new_length < current_length:
                    current_tour = new_tour
                    current_length = new_length
                    improved = True

    return current_tour

# Find the TSP tour using the 2-Opt algorithm
tsp_tour_2opt = two_opt_algorithm(G)

# Visualize the 2-Opt TSP tour
image_with_tsp_2opt = image.copy()

# Draw the grid lines in a thin line
for x in range(0, 512, grid_cell_size):
    cv2.line(image_with_tsp_2opt, (x, 0), (x, 512), 128, 1)
    cv2.line(image_with_tsp_2opt, (0, x), (512, x), 128, 1)

# Draw the vertex weights as black text labels
for i in range(len(center_points)):
    x, y = center_points[i]
    
    weight = int(G.nodes[i]['weight'])
    cv2.putText(image_with_tsp_2opt, str(weight), (x, y), cv2.FONT_HERSHEY_SIMPLEX, 0.5, 0, 1)

# Draw the rectilinear edges in a thick white line
for i in range(len(tsp_tour_2opt) - 1):
    node1 = tsp_tour_2opt[i]
    node2 = tsp_tour_2opt[i + 1]
    draw_rectilinear_line(image_with_tsp_2opt, center_points[node1], center_points[node2], 255, 2)

In [80]:
def tour_weight(tour, graph):
    weight = 0
    for i in range(len(tour) - 1):
        weight += graph[tour[i]][tour[i + 1]]['weight']
    return weight

def mst_weight(mst):
    return sum(weight['weight'] for u, v, weight in mst.edges(data=True))

mst_total_weight = mst_weight(mst)
print("Combined weight of the Minimum Spanning Tree:", mst_total_weight)

combined_weight = tour_weight(tsp_tour_2opt, G)
print("Combined weight of the optimal road network using 2-Opt TSP:", combined_weight)

combined_weight = tour_weight(tsp_tour, G)
print("Combined weight of the optimal road network using TSP:", combined_weight)

Combined weight of the Minimum Spanning Tree: 2130
Combined weight of the optimal road network using 2-Opt TSP: 2610
Combined weight of the optimal road network using TSP: 4200


In [81]:
# Display the images
cv2.imshow("Original Image", image)
cv2.imshow("Image with Graph", image_with_graph)
cv2.imshow("Image with Graph and Weights", image_with_graph_weights)
cv2.imshow("Image with MST", image_with_mst)
cv2.imshow("Image with TSP", image_with_tsp)
cv2.imshow("Image with 2OPT", image_with_tsp_2opt)


cv2.waitKey(0)
cv2.destroyAllWindows()

# Save the images
# cv2.imwrite("original_image.png", image)
# cv2.imwrite("image_with_graph.png", image_with_graph)
cv2.imwrite("image_with_graph_weights.png", image_with_graph_weights)
cv2.imwrite("image_with_mst.png", image_with_mst)
cv2.imwrite("image_with_tsp.png", image_with_tsp)
cv2.imwrite("image_with_2opt_tsp.png", image_with_tsp_2opt)

True

In [82]:
tsp_tour_2opt

[0,
 17,
 11,
 7,
 21,
 12,
 16,
 22,
 3,
 8,
 20,
 14,
 10,
 18,
 9,
 2,
 5,
 6,
 15,
 1,
 19,
 4,
 13,
 23]